# VGG16 Model

In [9]:
# Imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from city_sustainability.models import unet_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.quality import life_quality
from tensorflow.keras.applications.vgg16 import VGG16
from city_sustainability.loading_data import loading_paths
from city_sustainability.preprocessing import image_resize
from tensorflow.keras.applications.vgg16 import preprocess_input
from city_sustainability.loading_data import image_and_label_arrays
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, UpSampling2D, Concatenate, Input

In [ ]:
data_path ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths, label_paths = loading_paths(data_path)

image_datagen = ImageDataGenerator(validation_split=0.2)
label_datagen = ImageDataGenerator(validation_split=0.2)

seed = 28
batch_size = 64

image_df = pd.DataFrame(image_paths_448, columns=["filename"])
label_df = pd.DataFrame(label_paths_448, columns=["filename"])

image_generator = image_datagen.flow_from_dataframe(
    dataframe=image_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(448, 448),
    subset = 'training',
    shuffle=False)

label_generator = label_datagen.flow_from_dataframe(
    dataframe=label_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(448, 448),
    subset = 'training',
    shuffle=False)

validation_image_generator = image_datagen.flow_from_dataframe(
    dataframe=image_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(448, 448),
    subset = 'validation',
    shuffle=False)

validation_label_generator = label_datagen.flow_from_dataframe(
    dataframe=label_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(448, 448),
    subset = 'validation',
    shuffle=False)

train_generator = zip(image_generator, label_generator)
validation_generator = zip(validation_image_generator, validation_label_generator)

def preprocess_labels(labels):
    categorical_labels = to_categorical(labels, num_classes=9)
    return categorical_labels

train_generator = ((images, preprocess_labels(labels)) for images, labels in train_generator)
validation_generator = ((images, preprocess_labels(labels)) for images, labels in validation_generator)

In [ ]:
# Create the model
model_full = unet_model.build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

In [ ]:
# Total params: 26,482,249
# Trainable params: 11,764,617
# Non-trainable params: 14,717,632
model_full.summary()

In [ ]:
# Compile the model
unet_model.compile_model(model_full)

In [ ]:
# Train the model on the current batch
model_full.fit(train_generator, 
               epochs=50,
               steps_per_epoch= image_generator.samples//batch_size,
               validation_data= validation_generator,
               validation_steps= validation_image_generator.samples//batch_size)

In [ ]:
model_full.save('../../model_training/marcel/20230529-06-unet_vgg16full_1.00img_50epch_64btch')

In [ ]:
# Train the model on the current batch
model_full.fit(train_generator, 
               epochs=1,
               steps_per_epoch= image_generator.samples//batch_size,
               validation_data= validation_generator,
               validation_steps= validation_image_generator.samples//batch_size)

In [ ]:
# Evaluate Model Full
unet_model.evaluate_model(model_full, X_test_scaled, y_test)

# EfficientNet-B4 Model

In [17]:
# Retrieve image_paths and label_paths
data_path ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths, label_paths = loading_paths(data_path)

# Import all image array as X and label array as y
X, y = image_and_label_arrays(image_paths, label_paths, sampling_ratio=0.33)

# Apply holdout
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Scale X and y
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [21]:
X_train_scaled.shape

(620, 256, 256, 3)

In [22]:
X_test_scaled.shape

(267, 256, 256, 3)

In [18]:
# Set the batch size and number of epochs
epochs = 100
num_classes = 9
batch_size = 32
input_shape = (256, 256, 3)

# Load the EfficientNetB4 model
model_effnet = EfficientNetB4(include_top=False, weights='imagenet', input_shape=input_shape, classes=num_classes)

# Compile the model
unet_model.compile_model(model_effnet)

# Print the model summary
model_effnet.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_8 (Rescaling)        (None, 256, 256, 3)  0           ['input_5[0][0]']                
                                                                                                  
 normalization_4 (Normalization  (None, 256, 256, 3)  7          ['rescaling_8[0][0]']            
 )                                                                                                
                                                                                     

                                                                                                  
 block2a_expand_conv (Conv2D)   (None, 128, 128, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 128, 128, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 128, 128, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_d

 block2c_expand_activation (Act  (None, 64, 64, 192)  0          ['block2c_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 64, 64, 192)  1728       ['block2c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2c_bn (BatchNormalization  (None, 64, 64, 192)  768        ['block2c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2c_activation (Activation  (None, 64, 64, 192)  0          ['block2c_bn[0][0]']             
 )        

                                                                                                  
 block3a_dwconv (DepthwiseConv2  (None, 32, 32, 192)  4800       ['block3a_dwconv_pad[0][0]']     
 D)                                                                                               
                                                                                                  
 block3a_bn (BatchNormalization  (None, 32, 32, 192)  768        ['block3a_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3a_activation (Activation  (None, 32, 32, 192)  0          ['block3a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3a_s

 block3c_se_squeeze (GlobalAver  (None, 336)         0           ['block3c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block3c_se_reshape (Reshape)   (None, 1, 1, 336)    0           ['block3c_se_squeeze[0][0]']     
                                                                                                  
 block3c_se_reduce (Conv2D)     (None, 1, 1, 14)     4718        ['block3c_se_reshape[0][0]']     
                                                                                                  
 block3c_se_expand (Conv2D)     (None, 1, 1, 336)    5040        ['block3c_se_reduce[0][0]']      
                                                                                                  
 block3c_se_excite (Multiply)   (None, 32, 32, 336)  0           ['block3c_activation[0][0]',     
          

...

 block6b_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6b_se_reshape[0][0]']     
                                                                                                  
 block6b_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block6b_se_reduce[0][0]']      
                                                                                                  
 block6b_se_excite (Multiply)   (None, 8, 8, 1632)   0           ['block6b_activation[0][0]',     
                                                                  'block6b_se_expand[0][0]']      
                                                                                                  
 block6b_project_conv (Conv2D)  (None, 8, 8, 272)    443904      ['block6b_se_excite[0][0]']      
                                                                                                  
 block6b_project_bn (BatchNorma  (None, 8, 8, 272)   1088        ['block6b_project_conv[0][0]']   
 lization)

                                                                  'block6d_se_expand[0][0]']      
                                                                                                  
 block6d_project_conv (Conv2D)  (None, 8, 8, 272)    443904      ['block6d_se_excite[0][0]']      
                                                                                                  
 block6d_project_bn (BatchNorma  (None, 8, 8, 272)   1088        ['block6d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6d_drop (Dropout)         (None, 8, 8, 272)    0           ['block6d_project_bn[0][0]']     
                                                                                                  
 block6d_add (Add)              (None, 8, 8, 272)    0           ['block6d_drop[0][0]',           
          

 lization)                                                                                        
                                                                                                  
 block6f_drop (Dropout)         (None, 8, 8, 272)    0           ['block6f_project_bn[0][0]']     
                                                                                                  
 block6f_add (Add)              (None, 8, 8, 272)    0           ['block6f_drop[0][0]',           
                                                                  'block6e_add[0][0]']            
                                                                                                  
 block6g_expand_conv (Conv2D)   (None, 8, 8, 1632)   443904      ['block6f_add[0][0]']            
                                                                                                  
 block6g_expand_bn (BatchNormal  (None, 8, 8, 1632)  6528        ['block6g_expand_conv[0][0]']    
 ization) 

                                                                  'block6g_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 8, 8, 1632)   443904      ['block6h_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 8, 8, 1632)  6528        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 8, 8, 1632)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_d

__________________________________________________________________________________________________


In [19]:
# This line runs the model without batches.
unet_model.train_model(model_effnet, X_train_scaled, y_train, batch_size = batch_size, epochs = 1, validation_split=0.2)

ValueError: in user code:

    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 11

In [ ]:
#Save Model Full
model_effnet.save('../../model_training/marcel')

In [ ]:
#Evaluate Model Full
unet_model.evaluate_model(model_effnet, X_test_scaled, y_test)